In [ ]:
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona

driver = gdal.GetDriverByName('GTiff')
#rasterFileName = "/home/gadiraju/data/bl-slums/raw-img/MUL_mosaic_415.tif" #path to raster
rasterFileName1 = "/scratch/slums/bl-slums/raw-img/landsat7/PS-feb-2002-extract.tif"
dataset1 = gdal.Open(rasterFileName1)
rasterFileName2 = "/scratch/slums/bl-slums/raw-img/landsat7/ETM-feb-2002-extract-NDBI.tif"
dataset2 = gdal.Open(rasterFileName2)
rasterFileName3 = "/scratch/slums/bl-slums/raw-img/landsat7/simple-feb-2002-extract.tif"
dataset3 = gdal.Open(rasterFileName3)
rasterFileName4 = "/scratch/slums/bl-slums/raw-img/landsat7/adv-feb-2002-extract.tif"
dataset4 = gdal.Open(rasterFileName4)
rasterFileName5 = "/scratch/slums/bl-slums/raw-img/landsat7/higher-feb-2002-extract.tif"
dataset5 = gdal.Open(rasterFileName5)
vector = fiona.open('/scratch/slums/bl-slums/gt/final-data/landsat-final/gt10e/gt10e.shp')
imggeotrans1 = dataset1.GetGeoTransform()
coordinates_list = []
count_numbers = [0]*2
train_or_test = []
#print count_numbers
actual_class = []
for feat in vector:
    curr_class = feat['properties']['ULabel']
    
    if curr_class >0:
        #print curr_class
        #print curr_class
        train_or_test.append(feat['properties']['Type'])
        coordinates_list.append(feat['geometry']['coordinates'])
        #count_numbers[curr_class-1]+=1
        actual_class.append(curr_class)

bands1=[]
bands2=[]
bands3=[]
bands4=[]
bands5=[]
data_all_bands = []
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize

transform = dataset1.GetGeoTransform()

imggeotrans2 = dataset2.GetGeoTransform()
imggeotrans3 = dataset3.GetGeoTransform()


print '{} \n +++++++++++++++++++++++++++ \n {} \n {}'.format(imggeotrans1, imggeotrans2, imggeotrans3)

xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

print xOrigin, yOrigin, pixelWidth, pixelHeight

for i in range(6):
    bands1.append(dataset1.GetRasterBand(i+1)) 
    # data_all_bands.append(band.ReadAsArray(0,0,cols,rows).astype(np.float))
bands2.append(dataset2.GetRasterBand(1))
for i in range(8):#simple
    bands3.append(dataset3.GetRasterBand(i+1)) 
for i in range(10):#adv
    bands4.append(dataset4.GetRasterBand(i+1)) 
for i in range(11):#higher
    bands5.append(dataset5.GetRasterBand(i+1)) 
points_list = coordinates_list #list of X,Y coordinates



In [ ]:
print points_list

In [7]:
#points_list = [(756073.458902 , 1456683.91481)]
train_images_list = [[],[]]
test_images_list = [[],[]]
coords_list = [[],[]]
for pt in range(len(points_list)):
    if pt%500 == 0:
        print 'Finished {} points\n'.format(pt)
    point = points_list[pt]
    cls = actual_class[pt]
    #print point, cls
    curr = np.zeros((1,36))-1
    curr = curr.astype(float)
    col = int((point[0] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[1] ) / pixelHeight)
    for k in range(6):
        data = bands1[k].ReadAsArray(col,row,1,1).astype(np.float)
        #print point[0], point[1], data
        curr[0,k] = data
    data = bands2[0].ReadAsArray(col,row,1,1).astype(np.float)
    curr[0,6] = data
    for k in range(8):
        data = bands3[k].ReadAsArray(col,row,1,1).astype(np.float)
        curr[0,7+k] = data 
    for k in range(10):
        data = bands4[k].ReadAsArray(col,row,1,1).astype(np.float)
        curr[0,15+k] = data
    for k in range(11):
        data = bands5[k].ReadAsArray(col,row,1,1).astype(np.float)
        curr[0,25+k] = data
    if train_or_test[pt] == 1:
        train_images_list[cls-1].append(curr)
    else:
        test_images_list[cls-1].append(curr)
        
print len(train_images_list[0]), len(train_images_list[1]), len(test_images_list[0]), len(test_images_list[1])

Finished 0 points

Finished 500 points

Finished 1000 points

Finished 1500 points

Finished 2000 points

Finished 2500 points

1196 1125 242 282


In [8]:
for i in range(2):
    curr_train_imgs = train_images_list[i]
    n_train_images = len(curr_train_imgs)
    print n_train_images
    curr_test_imgs = test_images_list[i]
    n_test_images = len(curr_test_imgs)
    print n_test_images
    curr_train_y = np.zeros((n_train_images, 1))+i
    curr_test_y = np.zeros((n_test_images, 1))+i
    if i == 0:
        trainX = np.asarray(curr_train_imgs)
        trainY = curr_train_y
        testX = np.asarray(curr_test_imgs)
        testY = curr_test_y
        
    else:
        trainX = np.vstack((trainX, curr_train_imgs))
        trainY = np.vstack((trainY, curr_train_y))
        testX = np.vstack((testX, curr_test_imgs))
        testY = np.vstack((testY, curr_test_y))

trainX = trainX.reshape(trainX.shape[0],trainX.shape[2])
testX = testX.reshape(testX.shape[0], testX.shape[2])
print trainX.shape, trainY.shape, testX.shape, testY.shape

1196
242
1125
282
(2321, 36) (2321, 1) (524, 36) (524, 1)


In [9]:
import pickle

f = open('/scratch/slums/bl-slums/gt/LS-final-px-tr-2-Xa','w')
pickle.dump(trainX,f)
f.close()

f = open('/scratch/slums/bl-slums/gt/LS-final-px-tr-2-Ya','w')
pickle.dump(trainY,f)
f.close()


f = open('/scratch/slums/bl-slums/gt/LS-final-px-te-2-Xa','w')
pickle.dump(testX,f)
f.close()

f = open('/scratch/slums/bl-slums/gt/LS-final-px-te-2-Ya','w')
pickle.dump(testY,f)
f.close()